In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import branca.colormap as cm
from loguru import logger

In [ ]:
sys.path.append("../")  # include parent directory
from src.pond_data import PondDataset, PondDataModule
from src.config_utils import build_kwargs_from_config

## Visualize Model Tiles for Cloud Cover Check

This notebook creates a folium map of model tiles with estimated level of cloud cover. This is so we can find which model tiles have annotations when in fact they are covered in clouds. One we identify the tiles that are too cloudy and have annotations, we do the following
1. Keep the image for the model but revise its corresponding annotation mask to be all background
2. Ignore both the image its corresponding annotation mask.

## Input
- Satellite Images (geoTIFF files)
- Raster Masks (geoTIFF files)

## Output
- Folium map of model grid cells colored by estimated level of cloud cover (folium HTML file)

## Set up parameters from config yaml

Feel free to edit the parameters here as well

In [ ]:
DATA_PATH = Path("../data/")
CONFIG_PATH = Path("../config")

CONFIG_FPATH = CONFIG_PATH / "pond_config.yaml"

STATIC_MAP_DIR = DATA_PATH / "static_maps"
STATIC_MAP_DIR.mkdir(exist_ok=True)

In [ ]:
kwargs_dict = build_kwargs_from_config(DATA_PATH, CONFIG_FPATH)

In [ ]:
DATASET_KWARGS = kwargs_dict["dataset_kwargs"]
DATASET_KWARGS

## Setup Planet API Key

In [ ]:
planet_config_path = CONFIG_PATH / "secrets/planet_config.json"
with open(planet_config_path) as file:
    planet_config = json.load(file)
planet_xyz_url = "https://tiles3.planet.com/basemaps/v1/planet-tiles/planet_medres_normalized_analytic_2022-07_mosaic/gmap/{z}/{x}/{y}.png?api_key="
planet_xyz_url = planet_xyz_url + planet_config["PLANET_API_KEY"]

## Setting up the Pytorch Dataset

In [ ]:
%%time
pond_dataset = PondDataset(**DATASET_KWARGS)
pond_dataset

In [ ]:
# check labels
pond_dataset.label_mapping

In [ ]:
# check a sample from the dataset
i = 2
pond_dataset[i]

## Get a geodataframe of tile bounding boxes with median pixel vals

We get the median pixel value across all the images. This value is our proxy for cloud cover because high pixel values correspond to having more "whiteness" in the image.

In [ ]:
%%time
median_pixel_val = pond_dataset.get_percentile_pixel_values(percentile=50)

In [ ]:
%%time
tile_bboxes = pond_dataset.get_tile_bboxes()

In [ ]:
cloud_check_gdf = {"median_pixel_val": median_pixel_val, "geometry": tile_bboxes}
cloud_check_gdf = pd.DataFrame.from_dict(cloud_check_gdf)
cloud_check_gdf.index.name = "quadkey"
cloud_check_gdf = cloud_check_gdf.reset_index()
cloud_check_gdf["median_pixel_val_log"] = cloud_check_gdf["median_pixel_val"].apply(
    np.log
)
cloud_check_gdf.head()

In [ ]:
cloud_check_gdf["geometry"] = gpd.GeoSeries(
    cloud_check_gdf["geometry"], crs=pond_dataset.crs
)
cloud_check_gdf = gpd.GeoDataFrame(
    cloud_check_gdf, geometry="geometry", crs=pond_dataset.crs
)
cloud_check_gdf = cloud_check_gdf.to_crs("epsg:4326")
cloud_check_gdf.head()

## Plot histogram of pixel values

In [ ]:
cloud_check_gdf["median_pixel_val"].hist(bins=100, legend=True)
plt.show()

In [ ]:
cloud_check_gdf["median_pixel_val_log"].hist(bins=100, legend=True)
plt.show()

## Create the folium map

In [ ]:
pixel_val_col = "median_pixel_val_log"
vmin = cloud_check_gdf[pixel_val_col].min()
vmax = cloud_check_gdf[pixel_val_col].max()

colormap = cm.LinearColormap(colors=["Green", "Yellow", "Red"], vmin=vmin, vmax=vmax)

In [ ]:
%%time
m = folium.Map(tiles=None)

folium.TileLayer(tiles=planet_xyz_url, attr="NICFI", name="NICFI").add_to(m)
folium.TileLayer(tiles="OpenStreetMap", name="OSM").add_to(m)

m = cloud_check_gdf.explore(
    m=m,
    column=pixel_val_col,
    cmap=colormap,
    name="Model Tiles",
    popup=["quadkey"],
    tooltip=[pixel_val_col],
)

# Fit map to bounds
minx, miny, maxx, maxy = cloud_check_gdf.total_bounds
m.fit_bounds([[miny, minx], [maxy, maxx]])

folium.LayerControl().add_to(m)
folium.ClickForMarker().add_to(m)

## Save Map as HTML

In [ ]:
%%time
parent_dir = kwargs_dict["misc_kwargs"]["parent_dir"]
map_fname = f"ci-cloud-check-map-{parent_dir}.html"
map_fpath = STATIC_MAP_DIR / map_fname
m.save(map_fpath)

logger.info(f"Saved map to {map_fpath}")